# 知识工程-作业5 潜在语义索引构建

2024214500 叶璨铭


## 代码与文档格式说明

> 本文档使用Jupyter Notebook编写，所以同时包括了实验文档和实验代码。

> 本次实验项目采用了类似于 Quarto + nbdev 的方法来同步Jupyter Notebook代码到python文件, 因而我们的实验文档导出为pdf和html格式可以进行阅读，而我们的代码也导出为python模块形式，可以作为代码库被其他项目使用。
我们这样做的好处是，避免单独管理一堆 .py 文件，防止代码冗余和同步混乱，py文件和pdf文件都是从.ipynb文件导出的，可以保证实验文档和代码的一致性。

> 本文档理论上支持多个格式，包括ipynb, html, docx, pdf, md 等，但是由于 quarto和nbdev 系统的一些bug，我们目前暂时只支持ipynb文件，以后有空的时候解决bug可以构建一个[在线文档网站](https://thu-coursework-machine-learning-for-big-data-docs.vercel.app/)。您在阅读本文档时，可以选择您喜欢的格式来进行阅读，建议您使用 Visual Studio Code (或者其他支持jupyter notebook的IDE, 但是VSCode阅读体验最佳) 打开 `ipynb`格式的文档来进行阅读。


> 为了记录我们自己修改了哪些地方，使用git进行版本控制，这样可以清晰地看出我们基于助教的代码在哪些位置进行了修改，有些修改是实现了要求的作业功能，而有些代码是对助教的代码进行了重构和优化。我将我在知识工程课程的代码，在作业截止DDL之后，开源到 https://github.com/2catycm/THU-Coursework-Knowledge-Engineering.git ，方便各位同学一起学习讨论。


## 代码规范说明

为了让代码清晰规范，在作业开始前，使用 `ruff format`格式化助教的代码; 在我们实现函数过程中，函数的docstring应当遵循fastai规范而不是numpy规范，这样简洁清晰，不会Repeat yourself。

```bash
❯ ruff format
error: Failed to parse util.py:5:34: Multiple return types must be parenthesized
2 files reformatted
```

原来，老师给的`util.py:5:34`代码这里类型标注不规范，稍作修改即可
```python
-def load_words(filepath: str) -> List[str], List[List[str]]:
+def load_words(filepath: str) -> Tuple[List[str], List[List[str]]]:
```

此外，原本代码的类型不严谨，语义不规范，导致VSCode报了很多错，我们先重构一下助教的代码，增加合适的注释和类型提示。

清晰的类型注解也是能够帮助我们更好的理解代码的，提高我们对作业的理解，所以不惜花一点时间。


比如 import * 不是很规范。

![alt text](image.png)

我们简单修复一下即可

```python
-from typing import *
+from typing import Tuple, List
```

原本的代码多次错误地使用了 `np.array` 来标注类型，这是不对的，应该使用 `np.ndarray` 来标注类型。比如这个地方

```python
-def cal_tfidf_matrix(term_doc: np.array, documents: List[List[str]], terms: List[str]):
+def cal_tfidf_matrix(term_doc: np.ndarray, documents: List[List[str]], terms: List[str]):
```

此外这个地方应该用多了, 实际上我们这次作业不需要这个标准库，而且python3.12不建议用。
```python
-import imp
```

```bash
/home/ye_canming/repos/assignments/THU-Coursework-Knowledge-Engineering/5.潜在语义索引构建的作业/main.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
```


这下我们修改完，pylance不再报错了，可以安心写主要逻辑了。


## 原理回顾和课件复习



## 数据加载



我们看下 main.py 文件

使用到数据的地方是

```python
terms_tmp, documents_tmp = load_words(os.path.join(filePath, file))

```

## 运行结果

In [1]:
!python main.py

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.540 seconds.
Prefix dict has been built successfully.
TOP10单词的TF-TDF如下所示
 [('阿富汗', 0.006327252874692564), ('欧洲杯', 0.005215631936116387), ('折叠', 0.0050477958652063015), ('意大利', 0.0050270258549414346), ('央行', 0.004862996707500968), ('任务', 0.004792787625878285), ('英国', 0.004705536630277911), ('戈贝尔', 0.004662253844871659), ('特鲁多', 0.004567633613773077), ('塔利班', 0.004254171083783253)]
U: (652, 652)
s: (652,)
VT: (23782, 23782)
[[0 5 3 2 0]
 [4 3 1 3 1]
 [1 2 1 1 1]
 [4 1 3 3 1]
 [1 4 3 3 1]]
查询关键词为: ['意大利封闭全国', '戈贝尔确诊新冠', '欧洲杯推迟', '汤姆汉克斯确诊', '特鲁多自我隔离', '疫情防控思政大课', '美国从阿富汗撤军', '英国央行紧急降息', '苹果折叠手机专利', '魔兽世界怀旧服']
查询结果top-5: [[ 13 392 203 147  18]
 [272 203 100 245  91]
 [100 147  64  90  57]
 [272 100 203 245 111]
 [100 272 203 245  90]
 [392 147  13 132 401]
 [402 435 403 427 405]
 [486 482 485 506 468]
 [520 518 539 540 519]
 [575 650 637 632 641]]
查询结果top-5准确率: [0.4, 0.4, 0.2,